You need to restart runtime after the installation otherwise it will not work.

In [ ]:
!pip3 install pytextrank
!spacy download en_core_web_sm
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 47 kB 5.6 MB/s 
     |████████████████████████████████| 1.1 MB 53.2 MB/s 
     |████████████████████████████████| 33.8 MB 488 kB/s 
     |████████████████████████████████| 9.4 MB 57.8 MB/s 
     |████████████████████████████████| 965 kB 61.7 MB/s 
     |████████████████████████████████| 295 kB 62.8 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
  Attempting uninstall: pygments
    Found existing installation: Pygments 2.6.1
    Uninstalling Pygments-2.6.1:
      Successfully uninstalled Pygments-2.6.1
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 457 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 24.1 MB/s 
     |████████████████████████████████| 7.6 MB 57.8 MB/s 
     |████████████████████████████████| 182 kB 72.3 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import spacy
from transformers.models.auto.processing_auto import AutoTokenizer
import torch
import pytextrank
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
import textwrap

tokenizer = AutoTokenizer.from_pretrained('t5-base')

model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/AIN311Project/Models/t5-base-question-generation_custom_20")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print ("device ",device)
model = model.to(device)

def keywordExt(text):
  # load a spaCy model, depending on language, scale, etc.
  nlp = spacy.load("en_core_web_sm")
  # add PyTextRank to the spaCy pipeline
  nlp.add_pipe("textrank")
  doc = nlp(text)
  # examine the top-ranked phrases in the document
  return doc._.phrases[:5]


def GenerateQuestion(text):
  encoding = tokenizer.encode_plus(text, return_tensors="pt")
  input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)
  output = greedy_decoding(input_ids,attention_masks)
  print (" \n ",output)

def greedy_decoding (inp_ids,attn_mask):
    greedy_output = model.generate(input_ids=inp_ids, attention_mask=attn_mask, max_length=256)
    Question =  tokenizer.decode(greedy_output[0], skip_special_tokens=True,clean_up_tokenization_spaces=True)
    return Question.strip().capitalize()

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
#@title Lets Generate Question
#@markdown Write the context in which you want AI to produce a question, if you want a specific question, also indicate the answer.

context = "Computer vision is an interdisciplinary scientific field that deals with how computers can gain high-level understanding from digital images or videos. From the perspective of engineering, it seeks to understand and automate tasks that the human visual system can do"  #@param
answer = '' #@param
#@markdown ---

if answer == '':
  keywords = keywordExt(context)
  answers = []
  for i in keywords:
    answers.append(i.text)

  lines = textwrap.wrap("\n%s\n" % context, width=100)
  print("\n".join(lines))
  print()
  print("Generated Questions: \n")
  for i in answers:
    text = "answer : " + i + "  " + "context : " + context
    GenerateQuestion(text)
    print()
else:
    text = "answer : " + answer + "  " + "context : " + context
    GenerateQuestion(text)
    print()

 Computer vision is an interdisciplinary scientific field that deals with how computers can gain
high-level understanding from digital images or videos. From the perspective of engineering, it
seeks to understand and automate tasks that the human visual system can do

Generated Questions: 

 
  What can computers gain high-level understanding from digital images or videos?

 
  What type of image do computers need to gain high-level understanding from?

 
  What is an interdisciplinary scientific field that deals with how computers can gain high-level understanding from digital images or videos?

 
  What is an interdisciplinary scientific field that deals with how computers can gain high-level understanding from digital images or videos?

 
  What does computer vision seek to understand and automate tasks that the human visual system can do?

